<a href="https://colab.research.google.com/github/sc21jr-fouritchhorse/JRGEM23/blob/main/Copy_of_SearchExercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0  59201      0 --:--:-- --:--:-- --:--:-- 59201


In [37]:
## A couple of possible goal states:
NORMAL_GOAL = [ [1,2,3],
                [4,5,6],
                [7,8,0] ]

SPIRAL_GOAL = [ [1,2,3],
                [8,0,4],
                [7,6,5] ]

### Declarations of some example states
LAYOUT_0 = [ [1,5,2],
             [0,4,3],
             [7,8,6] ]

LAYOUT_1 = [ [5,1,7],
             [2,4,8],
             [6,3,0] ] 

LAYOUT_2 = [ [2,6,3],
             [4,0,5],
             [1,8,7] ] 

LAYOUT_3 = [ [7,2,5],
             [4,8,1],
             [3,0,6] ] 

LAYOUT_4 = [ [8,6,7],  
             [2,5,4],
             [3,0,1] ] 

In [38]:
def number_position_in_layout( n, layout):
    for i, row in enumerate(layout):
        for j, val in enumerate(row):
            if val==n:
                return (i,j)

In [39]:
from copy import deepcopy

class EightPuzzle( SearchProblem ):
        
    action_dict = {
        (0,0) : [(1, 0, 'up'),    (0, 1, 'left')],
        (0,1) : [(0, 0, 'right'), (1, 1, 'up'),    (0, 2, 'left')],
        (0,2) : [(0, 1, 'right'), (1, 2, 'up')],
        (1,0) : [(0, 0, 'down'),  (1, 1, 'left'),  (2, 0, 'up')],
        (1,1) : [(1, 0, 'right'), (0, 1, 'down'),  (1, 2, 'left'), (2, 1, 'up')],
        (1,2) : [(0, 2, 'down'),  (1, 1, 'right'), (2, 2, 'up')],
        (2,0) : [(1, 0, 'down'),  (2, 1, 'left')],
        (2,1) : [(2, 0, 'right'), (1, 1, 'down'),  (2, 2, 'left')],
        (2,2) : [(2, 1, 'right'), (1, 2, 'down')]
    }
    
    
    def __init__(self, initial_layout, goal_layout ):
        pos0 = number_position_in_layout( 0, initial_layout )
        # Initial state is pair giving initial position of space
        # and the initial tile layout.
        self.initial_state = ( pos0, initial_layout)
        self.goal_layout   = goal_layout
        

    ### I just use the position on the board (state[0]) to look up the 
    ### appropriate sequence of possible actions.
    def possible_actions(self, state ):
        actions =  EightPuzzle.action_dict[state[0]]
        actions_with_tile_num = []
        for r, c, d in actions:
            tile_num = state[1][r][c] ## find number of moving tile
            # construct the action representation including the tile number
            actions_with_tile_num.append( (r, c, (tile_num,d)) )
        return actions_with_tile_num

    def successor(self, state, action):
        old0row, old0col  =  state[0]    # get start position
        new0row, new0col, move = action  # unpack the action representation
        moving_number, _ = move
        ## Make a copy of the old layout
        newlayout = deepcopy(state[1])
        # Swap the positions of the new number and the space (rep by 0)
        newlayout[old0row][old0col] = moving_number
        newlayout[new0row][new0col] = 0
        return ((new0row, new0col), newlayout )
    
    def goal_test(self,state):
        return state[1] == self.goal_layout
    
    def display_action(self, action):
        _,_, move = action
        tile_num, direction = move
        print("Move tile", tile_num, direction)
        
    def display_state(self,state):
        for row in state[1]:
            nums = [ (n if n>0 else '.') for n in row]
            print( "   ", nums[0], nums[1], nums[2] )

In [40]:
EP = EightPuzzle( LAYOUT_1, NORMAL_GOAL )

In [41]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/crazy8heuristics.py > bbmodcache/crazy8heuristics.py
from bbmodcache.crazy8heuristics import bb_misplaced_tiles, bb_manhattan

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4731  100  4731    0     0  21120      0 --:--:-- --:--:-- --:--:-- 21120


In [42]:
def cost(path, state):
        return len(path)



In [43]:
## Specify and execute an A* search run here:




Random 8 puzzles:
[[6,2,0],[7,5,3],[4,8,1]]
[[7,4,8],[6,3,1],[2,0,5]]
[[2,5,8],[3,4,1],[7,6,0]]
[[6,5,4],[7,3,2],[0,1,8]]
[[5,1,6],[4,2,8],[0,7,3]]

One away from solution
[[1,2,3],[4,5,6],[7,0,8]]
[[1,2,3],[4,5,0],[7,8,6]]
[[1,0,3],[8,2,4],[7,6,5]]
[[1,2,3],[8,4,0],[7,6,5]]

Three away from solution
[[1,2,3],[0,4,6],[7,5,8]]
[[1,0,2],[4,5,3],[7,8,6]]
[[2,0,3],[1,8,4],[7,6,5]]
[[1,2,3],[8,4,0],[7,6,5]]

The goal
[[1,2,3],[4,5,6],[7,8,0]]
[[1,2,3],[8,0,4],[7,6,5]]

In [44]:
def insert_ep_data(data, is_spiral):
  random_puzzles = []
  for ep in data:
    random_puzzles.append(EightPuzzle(ep, NORMAL_GOAL if not is_spiral else SPIRAL_GOAL))
  return random_puzzles


In [45]:
import os 

def make_table_row(search_method, puzzles):
      output = ''
      output_stopper = open(os.devnull, 'w')
      for p in puzzles:
        for d in search(p,  search_method, 100, loop_check=True, dots=False, show_path=False, return_info=True).values():
          output += '{},'.format(d)
        output += '\n'
      output_stopper.close()
      print(output)
      return output
       

In [46]:
random_puzzle_data = [[[6,2,0],
                  [7,5,3],
                  [4,8,1]], 

                  [[7,4,8],
                  [6,3,1],
                  [2,0,5]], 

                  [[2,5,8],
                  [3,4,1],
                  [7,6,0]], 

                  [[6,5,4],
                  [7,3,2],
                  [0,1,8]], 

                  [[5,1,6],
                  [4,2,8],
                  [0,7,3]]]


one_from_solution_data = [
[[1,2,3],[4,5,6],[7,0,8]],
[[1,2,3],[4,5,0],[7,8,6]]]

one_from_spiral_data = [
[[1,0,3],[8,2,4],[7,6,5]],
[[1,2,3],[8,4,0],[7,6,5]]]

three_from_solution_data = [
[[1,2,3],[0,4,6],[7,5,8]],
[[1,0,2],[4,5,3],[7,8,6]]
]

three_from_spiral_data = [
[[2,0,3],[1,8,4],[7,6,5]],
[[1,2,3],[8,4,0],[7,6,5]]]


In [47]:
print(make_table_row('DF/LIFO', insert_ep_data(random_puzzle_data, False)))

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 100  (max number added to queue)
Search started (progress dot output off)

!! Search node limit (100) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =      158  (includes start)
Nodes discarded by loop_check  =       57  (101 distinct states added to queue)
Nodes tested (by goal_test)    =       55  (all expanded)
Nodes left in queue            =       45

Time taken = 0.0137 seconds

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 100  (max number added to queue)
Search started (progress dot output off)

!! Search node limit (100) reached !!
): No soluti